In [1]:
import time
start_time = time.time()

# Preparation - Load the packages and the data

Import the packages and load the data

In [2]:
# Import needed packages
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns; sns.set()
import glob
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import re
import nltk
from langdetect import detect
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import transformers
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer
from nbpep8.nbpep8 import pep8
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
#Set the working directory
os.chdir('C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\')

In [38]:
#Load not processed data
not_processed_df = pd.read_csv('Master Thesis Data\\merged_df.csv')

In [13]:
#Load the merged_df from the directory 'C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\Master Thesis Data\\':
merged_df = pd.read_csv('Master Thesis Data\\df_clean.csv')

# 3.Data Engineering

## 3.1 Add the tag-combinations

In [14]:
merged_df['tag_combined'] = merged_df.apply(lambda x: ''.join(sorted([str(x['tag_1']) if pd.notnull(x['tag_1']) else '', 
                                                         str(x['tag_2']) if pd.notnull(x['tag_2']) else ''])) , axis=1)


In [16]:
merged_df.head(1)

,Post_Creation_Date,Post_View_Count,Post_Title,Post_Score,Post_Answer_Count,Response_Scores,Post_Author,Answer_Author,Forum,Week_of_Year,tag_1,tag_2,tag_3,Post_Body_clean,tag_combined
0,2021-07-01 00:00:58,5867,"railsinstaller.org website was not found, what...",1,3,"1,0,0",8530236,8530236,Stack Overflow,26,ruby-on-rails,ssl,NaN,im taking course ruby rail video instructor re...,ruby-on-railsssl


## 3.2 Add date-time variables, treatment/control  and period indicators

In [17]:
#Week of year is already there

#Add column with weekly count of posts
merged_df['Weekly_Post_Count'] = merged_df.groupby('Week_of_Year')['Post_Title'].transform('count')

#Add D and T - dummy variables indicating whether the post was belongs to the control group or to the treatent group and whether it was created in the pre-treatment period in the treatment period (T)
merged_df['Post_Creation_Date'] = pd.to_datetime(merged_df['Post_Creation_Date'])
merged_df['D'] = np.where(merged_df['Post_Creation_Date'] < '2022-05-01', 0, 1)
merged_df['T'] = np.where(((merged_df['Post_Creation_Date'] >= '2021-07-01') & (merged_df['Post_Creation_Date'] < '2021-12-01')) | ((merged_df['Post_Creation_Date'] >= '2022-07-01') & (merged_df['Post_Creation_Date'] < '2022-12-01')), 0, 1)



# 3.3 Create the control variables

The following columns can be used as control variables:

-Post_View_Count

-Post_Score	

-Post_Answer_Count

-Mean Response_Scores (transform!)

-Question lenght (transform!)

	

In [7]:
merged_df.columns

Index(['Post_Creation_Date', 'Post_View_Count', 'Post_Title', 'Post_Score',
       'Post_Answer_Count', 'Response_Scores', 'Post_Author', 'Answer_Author',
       'Forum', 'Week_of_Year', 'tag_1', 'tag_2', 'tag_3', 'Post_Body_clean',
       'tag_combined', 'Weekly_Post_Count', 'D', 'T'],
      dtype='object')

In [32]:
def transform_response_scores(row):
    total = 0
    for part in row.split(','):
        try:
            total += int(part)
        except ValueError:
            # Part could not be converted to an integer, treat as 0
            total += 0
    return total

# Apply the function to the 'Response_Scores' column
merged_df['Response_Scores'] = merged_df['Response_Scores'].apply(transform_response_scores)


Add body and title lenght. For that we need orginal body

In [39]:
# Leave only index and 'Post_Body' column in not_processed_df
df_to_merge = not_processed_df[['Post_Body']]

In [42]:
df_to_merge

,Post_Body
0,<p>I'm taking a course on Ruby on Rails. In th...
1,<p>I have an ERP system which generates statem...
2,"<p>Given this example from <a href=""https://pl..."
3,<p>I have a folder structure like this:</p>\n<...
4,<p>In my Android Studio project I have a fragm...
...,...
3040725,"<p>When reading the <a href=""https://ethereum...."
3040726,<p>am building Dapp based on microservice and ...
3040727,"<p>so have a contract deployed already, and ET..."
3040728,<p>I have two contracts that implement <code>s...


In [41]:
#Merge merged_df with df_to_merge on index, left join
merged_df_2 = merged_df.merge(df_to_merge, how='left', left_index=True, right_index=True)

In [44]:
merged_df = merged_df_2 

In [45]:
def count_characters_title(row):
    if isinstance(row, str):
        return len(row)
    else:
        # Row is not a string, return 0
        return 0

def count_characters_body(row):
    if isinstance(row, str):
        return len(row)
    else:
        # Row is not a string, return 0
        return 0

# Apply the functions to the 'Post_Title' and 'Post_Body_clean' columns
merged_df['Title_Length'] = merged_df['Post_Title'].apply(count_characters_title)
merged_df['Body_Length'] = merged_df['Post_Body'].apply(count_characters_body)


In [46]:
merged_df

,Post_Creation_Date,Post_View_Count,Post_Title,Post_Score,Post_Answer_Count,Response_Scores,Post_Author,Answer_Author,Forum,Week_of_Year,...,tag_2,tag_3,Post_Body_clean,tag_combined,Weekly_Post_Count,D,T,Title_Length,Body_Length,Post_Body
0,2021-07-01 00:00:58,5867,"railsinstaller.org website was not found, what...",1,3,1,8530236,8530236,Stack Overflow,26,...,ssl,NaN,im taking course ruby rail video instructor re...,ruby-on-railsssl,30616,0,0,59,315,<p>I'm taking a course on Ruby on Rails. In th...
1,2021-07-01 00:01:10,11,Insert page break if MS Word document is endin...,0,0,0,3893947,0,Stack Overflow,26,...,NaN,NaN,erp system generates statement via word docx t...,ms-word,30616,0,0,65,718,<p>I have an ERP system which generates statem...
2,2021-07-01 00:01:18,1393,Plotly: How to show subplot titles at the bottom,1,1,2,2313889,13107804,Stack Overflow,26,...,plotly,subplot,given example plotly documentation displayed c...,plotlypython,30616,0,0,48,1095,"<p>Given this example from <a href=""https://pl..."
3,2021-07-01 00:01:28,660,Javascript files not loading in HTML page,1,2,1,15301811,4014834,Stack Overflow,26,...,html,d3.js,folder structure like referenced datajs appjs ...,htmljavascript,30616,0,0,41,7146,<p>I have a folder structure like this:</p>\n<...
4,2021-07-01 00:02:36,59,How can I put a fragment inside of a tab?,0,2,2,14595783,1440565,Stack Overflow,26,...,android,xml,android studio project fragment created displa...,androidjava,30616,0,0,41,12859,<p>In my Android Studio project I have a fragm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040725,2023-04-30 16:46:47,3,How do the LT and GT opcodes differ from SLT a...,0,1,0,116681,0,Ethereum,17,...,evm,opcode,reading opcodes evm article saw opcodes evalua...,evmsolidity,62331,1,1,66,441,"<p>When reading the <a href=""https://ethereum...."
3040726,2023-04-30 17:21:18,15,is it possible to give metamask a raw transact...,0,0,0,115531,0,Ethereum,17,...,web3.py,NaN,building dapp based microservice testing somet...,metamaskweb3.py,62331,1,1,83,1314,<p>am building Dapp based on microservice and ...
3040727,2023-04-30 21:02:45,41,Sending ETH to Gnosis Safe,0,0,0,98043,0,Ethereum,17,...,NaN,NaN,contract deployed already eth autosent wallet ...,gnosis-safe,62331,1,1,26,757,"<p>so have a contract deployed already, and ET..."
3040728,2023-04-30 22:03:50,11,How to override a function when two implementi...,0,1,0,37215,0,Ethereum,17,...,NaN,NaN,two contract implement one marked marked creat...,function-override,62331,1,1,91,1256,<p>I have two contracts that implement <code>s...


In [47]:
# #Save the merged_df to the directory 'C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\Master Thesis Data\\':
merged_df.to_csv('Master Thesis Data\\df_clean_transformed.csv', index=False)

In [48]:
end_time = time.time()
total_time = end_time - start_time
print("Total execution time: {:.2f} seconds".format(total_time))

Total execution time: 3729.06 seconds
